# Managing conversation history with the ChatGPT model
This sample notebook demonstrates a couple of simple patterns you can use for managing the prompts and conversation history with the ChatGPT model.

In [1]:
%env OPENAI_API_KEY=<YOUR KEY GOES HERE>

env: OPENAI_API_KEY='3fa9cc1420f14abbabd9382e5ae49634'


In [2]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade openai
%pip install jsonpointer==2.3 jsonschema==4.17.3 openai==0.27.2 tiktoken==0.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.27.2
    Uninstalling openai-0.27.2:
      Successfully uninstalled openai-0.27.2

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached openai-0.27.2-py3-none-any.whl (70 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.27.6
    Uninstalling openai-0.27.6:
      Successfully uninstalled openai-0.27.6

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# import os module & the OpenAI Python library for calling the OpenAI API
# please make sure you have installed required libraries via pip install -r requirements.txt
import os
import openai
import json

In [5]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)
    
# Setting up the deployment name
chatgpt_model_name = config_details['CHATGPT_MODEL'] 

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = config_details['OPENAI_API_BASE']

# Currently Chat Completions API have the following versions available: 2023-03-15-preview
openai.api_version = config_details['OPENAI_API_VERSION']

## 1.0 Create the system message for ChatGPT

In [6]:
base_system_message = "You are a helpful assistant.... Not"

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant.... Not


## 2.0 Define helper functions



In [7]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [7]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = openai.ChatCompletion.create(
        engine=chatgpt_model_name,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response['choices'][0]['message']['content']

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

## 3.0 Start the conversation

In [8]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I want to write a blog post about the impact of AI on the future of work."

In [9]:
# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [10]:
token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

Token count: 38


In [14]:
max_response_tokens = 500

response = send_message(messages, chatgpt_model_name, max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[SYSTEM]
You are a helpful assistant.... Not

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
Great topic! Here are some ideas to get you started:

1. Introduction: Introduce the topic and explain why it is important to discuss the impact of AI on the future of work.

2. The benefits of AI in the workplace: Discuss how AI can improve productivity, efficiency, and accuracy in various industries.

3. The potential risks and challenges of AI: Address concerns about job displacement, ethical issues, and the need for re-skilling and up-skilling.

4. The role of humans in a world with AI: Explore how humans can work alongside AI and the importance of developing skills that complement AI.

5. The future of work: Predict how AI will continue to shape the workplace and what changes we can expect to see in the coming years.

6. Conclusion: Sum up the main points of your post and offer some final thoughts on the impact of AI on the future of work.

Re

## 4.0 Continue the conversation

When working with the ChatGPT model, it's your responsibity to make sure you stay within the token limits of the model. The model can handle a maximum of 4096 tokens, and this includes the number of tokens in the prompt as well as the `max_tokens` you're requesting from the model. If you exceed these limits, the model will return an error.

You should also consider the trade-off between maintaining more of the conversation history and the cost/latency that you'll incur by including those tokens in the prompt. Shorter prompts are cheaper and faster. The amount of the previous conversation you include also makes a difference in how the model responds.

In this notebook, we'll show two strategies for managing the conversation history when working with the ChatGPT model.
- Option 1: Keep the conversation within a given token limit
- Option 2: Keep the conversation within a given number of turns

### Option 1: Keep the conversation within a given token limit

`overall_max_tokens` is the maximum number of tokens that you want to include in the prompt. Th maximum number this can be set to is 4096 but you can also consider reducing this number to reduce the cost and latency of the request.

In [15]:
overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [16]:
user_message = "The target audience for the blog post should be business leaders working in the tech industry."
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"role": "user", "content": user_message})

token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

# remove first message while over the token limit
while token_count > prompt_max_tokens:
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, chatgpt_model_name, max_response_tokens)

messages.append({"role": "assistant", "content": response})
print_conversation(messages)

Token count: 275
[SYSTEM]
You are a helpful assistant.... Not

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
Great topic! Here are some ideas to get you started:

1. Introduction: Introduce the topic and explain why it is important to discuss the impact of AI on the future of work.

2. The benefits of AI in the workplace: Discuss how AI can improve productivity, efficiency, and accuracy in various industries.

3. The potential risks and challenges of AI: Address concerns about job displacement, ethical issues, and the need for re-skilling and up-skilling.

4. The role of humans in a world with AI: Explore how humans can work alongside AI and the importance of developing skills that complement AI.

5. The future of work: Predict how AI will continue to shape the workplace and what changes we can expect to see in the coming years.

6. Conclusion: Sum up the main points of your post and offer some final thoughts on the impact of AI on the fu

### Option 2: Keep the conversation within a given number of turns

In [29]:
max_messages = 10

overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [31]:
user_message = "Make the post about generative AI aimed at business leaders who have some knowledge of the technology."
messages.append({"role": "user", "content": user_message})

while len(messages) > max_messages:
    messages.pop(0)

token_count = num_tokens_from_messages(messages)

while token_count > prompt_max_tokens:
    # remove first message from messages
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, chatgpt_model_name)
messages.append({"role": "assistant", "content": response})
print_conversation(messages)


[SYSTEM]
You are a helpful assistant.

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
Great idea! Here are some tips to get you started:

1. Define AI and explain how it works.
2. Discuss the current state of AI in the workplace and give examples of how it is being used.
3. Explore the potential impact of AI on jobs and industries.
4. Consider the ethical implications of AI and how it should be regulated.
5. Offer suggestions for how individuals and organizations can prepare for the future of work in an AI-driven world.

Remember to back up your claims with research and data, and make sure to write in a clear and concise manner that is easy for your readers to understand. Good luck!

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[ASSISTANT]
Great! In that case, you may want to tailor your writing to this specific audience. Here are some additional tips to consider:

1. Use industry-s